# Dutchess County Residental Sales Scraper

The code in this notebook will scrape data from https://gis.dutchessny.gov/parcelaccess/ to compile public records about residental sales in Dutchess County, NY from July 2019 - June 2022

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import json
from api_keys import api_keys


In [2]:
FROM_MONTH = '07'
FROM_YEAR = '2019'
TO_MONTH = '06'
TO_YEAR = '2022'

request_headers = {
    'accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding':'gzip, deflate, br',
    'accept-language':'en-US,en;q=0.8',
    'upgrade-insecure-requests':'1',
    'user-agent':'Mozilla/5.0 (Windows NY 10.0; Win64; x64) AppleWebKit/537.36(KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

#list of municipalities from dutchess county asp search page form
muni_ids = {
    132000:'Amenia',
    130200:'Beacon',
    132200:'Beekman',
    132400:'Clinton',
    132600:'Dover',
    132800:'East Fishkill',
    133089:'Fishkill',
    133200:'Hyde Park',
    133400:'Lagrange',
    133600:'Milan',
    133889:'North East',
    134089:'Pawling',
    134200:'Pine Plains',
    134400:'Pleasant Valley',
    134689:'Poughkeepsie',
    131300:'Poughkeepsie',
    134889:'Red Hook',
    135089:'Rhinebeck',
    135200:'Stanford',
    135400:'Union Vale',
    135689:'Wappinger',
    135889:'Washington',
    133001:'Fishkill',
    135801:'Millbrook',
    133801:'Millerton',
    134001:'Pawling',
    134801:'Red Hook',
    135001:'Rhinebeck',
    134803:'Tivoli',
    134601:'Wappingers Falls',
    135601:'Wappingers Falls'
}

#sublist of property classes from dutchess county asp search page form

property_classes = {
    '210 - One Family Year-Round Residence':210,
    '240 - Rural Residence with Acreage':240
}

url = 'https://gis.dutchessny.gov/parcelaccess/parcelaccessrescompsearch.asp'

In [3]:
def build_house_info_dicts(response):
    houses = []
    
    soup = BeautifulSoup(response.text)
    
    if len(soup.body.find_all('table')) > 1: #if there are any results from criteria
        results = soup.body.find_all('table')[-1].find_all('td') #select last table from results
        for i in range(len(results[::3])): #cycle through each 3 rows
            
            address_span = results[i*3].find_all('span')[1]
            address_info = []
            for row in address_span.children:
                address_info.append(row)
            address_info = address_info[::2]

            raw_info = []
            for row in results[(i*3)+2].children:
                raw_info.append(row)
            raw_info = raw_info[1:-4:2]

            #create dictionary for one residence
            house_info = {
                'address_street': address_info[0].strip(),
                'town': address_info[1].strip(),
                'parcel_id': address_info[2].strip(),
                'sale_date': raw_info[0].strip(),
                'sale_price': raw_info[1].replace('$','').strip(),
                'building_style': raw_info[2].strip(),
                'acreage': raw_info[3].strip(),
                'living_sqft':raw_info[4].strip(),
                'year_built':raw_info[5].strip(),
                'num_bedrooms':raw_info[6].strip(),
                'school_district':raw_info[7].strip(),
                'central_air': raw_info[8].strip(),
                'full_bathrooms': raw_info[9].split('/')[0].strip(),
                'half_bathrooms': raw_info[9].split('/')[1].strip(),
                'num_floors': raw_info[10].strip(),
                'grade': raw_info[11].strip(),
                'condition': raw_info[12].strip()
            }
            houses.append(house_info)
    
    return houses

In [4]:
all_homes = []
count = 0
#cycle through Municipalities and Property Classes

for muni in muni_ids.keys():
    
    for property_class in property_classes.values():
        
        params = {'muni':muni, #Municipality from muni_ids dictionary
                  'propclass':property_class, #Property class from property_classes dictionary
                  'buildingstyle':'18', #Any Building Style
                  'sqft1':'500', #Min Sq ft.
                  'sqft2':'100000', #Max Sq ft.
                  'lot1':'0.01', #Min Acreage
                  'lot2':'500', #Max Acreage
                  'year1':'1650', #Year Built, from
                  'year2':'2022', #Year Built, to
                  'bedrooms':'00', #Any number of bedroom
                  'frommonth': FROM_MONTH,
                  'fromyear': FROM_YEAR,
                  'tomonth': TO_MONTH,
                  'toyear': TO_YEAR
                 }
        
        #request.get for current params
        with requests.Session() as session:
            response = session.get(url, headers=request_headers, params=params)
            
        #build house info dictionary list from current search
        houses_from_page = build_house_info_dicts(response)
        
        #extend data to all_homes
        all_homes.extend(houses_from_page)
        
        print(f'{len(all_homes)} total houses found: {count+1} of {len(muni_ids)*len(property_classes)} pages loaded', end='\r')
        
        count += 1

In [5]:
import pandas as pd

houses_df = pd.DataFrame(all_homes)

In [6]:
houses_df.head()

,address_street,town,parcel_id,sale_date,sale_price,building_style,acreage,living_sqft,year_built,num_bedrooms,school_district,central_air,full_bathrooms,half_bathrooms,num_floors,grade,condition
0,16 Lavelle Rd,Amenia,7167-13-145335-0000,3/1/2022 11:20:46 AM,165000,Old style,1,945,1935,3,Webutuck CSD,No,1,0,1.7,Economy,Fair
1,7 Mountain View Rd,Amenia,7168-00-269071-0000,2/24/2022 1:05:54 PM,392000,Raised ranch,1.92,1344,1991,3,Webutuck CSD,Yes,2,0,1,Average,Normal
2,425 Leedsville Rd,Amenia,7267-00-407485-0000,2/18/2022 12:53:21 PM,190000,Ranch,0.58,1560,1960,3,Webutuck CSD,No,1,2,1,Average,Normal
3,350 Leedsville Rd,Amenia,7267-00-432304-0000,2/18/2022 12:51:49 PM,260000,Log home,1.29,838,1989,2,Webutuck CSD,No,1,0,1.5,Average,Normal
4,360 Bog Hollow Rd,Amenia,7264-00-244626-0000,2/15/2022 1:23:41 PM,353000,Ranch,0.61,1300,2020,3,Webutuck CSD,No,2,0,1,Average,Good


# Get Zipcode from Mapbox api

Using Mapbox's API, I will reverse lookup the zipcode for each home address.

In [7]:
api_key = api_keys['mapbox_api']

In [8]:
houses_df['zip'] = np.nan
houses_df['long'] = np.nan
houses_df['lat'] = np.nan

dutchess_co_zips = [12603, 12601, 12590, 12533, 12508, 12524, 12538, 12571, 12569, 12572, 12540, 12564, 12582, 12570, 12545, 12522, 12531, 12594, 12580, 12501, 12546, 12581, 12583, 12514, 12567, 12578, 12592, 12585, 12507, 12604, 12504, 12506, 12510, 12511, 12512, 12527, 12537, 12574, 12602]

def get_zip_lat_long(df): 
    count = 0
    null_count = 0
    for index, row in df.iterrows():
        address_string = f'{row.address_street}, {row.town}, NY, USA'

        url = f'https://api.mapbox.com/geocoding/v5/mapbox.places/{address_string}.json?country=us&limit=1&proximity=-73.79699289107144%2C41.68637000415529&types=address&access_token={api_key}'

        with requests.Session() as session:
            response = session.get(url)

        json_address = json.loads(response.text)
        try:
            context = json_address['features'][0]['context']
            for item in context:
                if item['text'].isnumeric():
                    if int(item['text']) in dutchess_co_zips:
                        zip_ = item['text']
                        break
            long = json_address['features'][0]['center'][0]
            lat = json_address['features'][0]['center'][1]
        except:
            pass
        
        df.loc[index, ['zip', 'lat', 'long']] = [zip_, lat, long]
        print(f'{count+1} / {len(df)}, {null_count} nulls passed', end='\r')
        count += 1
               
    return df
               

houses_df = get_zip_lat_long(houses_df) 

In [9]:
houses_df.head()

,address_street,town,parcel_id,sale_date,sale_price,building_style,acreage,living_sqft,year_built,num_bedrooms,school_district,central_air,full_bathrooms,half_bathrooms,num_floors,grade,condition,zip,long,lat
0,16 Lavelle Rd,Amenia,7167-13-145335-0000,3/1/2022 11:20:46 AM,165000,Old style,1,945,1935,3,Webutuck CSD,No,1,0,1.7,Economy,Fair,12501,-73.55693,41.845640
1,7 Mountain View Rd,Amenia,7168-00-269071-0000,2/24/2022 1:05:54 PM,392000,Raised ranch,1.92,1344,1991,3,Webutuck CSD,Yes,2,0,1,Average,Normal,12501,-73.55192,41.865780
2,425 Leedsville Rd,Amenia,7267-00-407485-0000,2/18/2022 12:53:21 PM,190000,Ranch,0.58,1560,1960,3,Webutuck CSD,No,1,2,1,Average,Normal,12501,-73.51019,41.849375
3,350 Leedsville Rd,Amenia,7267-00-432304-0000,2/18/2022 12:51:49 PM,260000,Log home,1.29,838,1989,2,Webutuck CSD,No,1,0,1.5,Average,Normal,12501,-73.50908,41.844660
4,360 Bog Hollow Rd,Amenia,7264-00-244626-0000,2/15/2022 1:23:41 PM,353000,Ranch,0.61,1300,2020,3,Webutuck CSD,No,2,0,1,Average,Good,12592,-73.51760,41.771060


In [10]:
houses_df['sale_date'] = pd.to_datetime(houses_df['sale_date'], infer_datetime_format=True)

numeric_features_int = ['year_built', 'num_bedrooms', 'full_bathrooms', 'half_bathrooms']
numeric_features_float = ['sale_price','acreage','living_sqft','num_floors', 'long', 'lat']

categorical_features = ['town','building_style', 'school_district', 'central_air', 'zip']
ordinal_features = ['grade','condition']

numeric_features_log = ['acreage','living_sqft']
numeric_features = ['year_built', 'num_bedrooms', 'full_bathrooms', 'half_bathrooms','num_floors', 'long', 'lat']

houses_df[numeric_features_int] = houses_df[numeric_features_int].astype('int')
houses_df[numeric_features_float] = houses_df[numeric_features_float].astype('float')
houses_df[categorical_features] = houses_df[categorical_features].astype('category')
houses_df[ordinal_features] = houses_df[ordinal_features].astype('category')


In [11]:
import pyarrow.feather as feather

feather.write_feather(houses_df, 'data/dutchess_county_residence_sales_07_2019_06_2022.feather')